# Interacting with Kinetica vector similarity search via LLM
### Install required packages

In [ ]:
!pip3.10 install gpudb==7.2.0.0b0 pandas pyarrow typeguard langchain langchain_openai nemollm colorlog langchain-kinetica

In [ ]:
import kinetica.setup
kinetica.setup.setup()

### Connect to Kinetica and the LLM

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_kinetica import KineticaChatLLM, KineticaSqlOutputParser, SqlResponse
from IPython.display import display, HTML

# create the Kinetica connection
kdbc = KineticaChatLLM._create_kdbc(host="https://demo72.kinetica.com/_gpudb", login="gtc", password="Kinetica123!")

# create the Kinetica LLM
kinetica_llm = KineticaChatLLM(kdbc=kdbc)

# Set the context to use
kinetica_ctx = 'nyse.nyse_vector_ctxt'

Connected to Kinetica: http://172.31.33.30:9191. (api=7.2.0.0b0, server=7.2.0.1)


### Set up the context

In [7]:
# load the context from the database
ctx_messages = kinetica_llm.load_messages_from_context(kinetica_ctx)

# Add the input prompt. This is where input question will be substituted.
ctx_messages.append(("human", "{input}"))

# Create the prompt template.
prompt_template = ChatPromptTemplate.from_messages(ctx_messages)
prompt_template.pretty_print()

# create the chain. 
# note: The KineticaSqlOutputParser will execute the SQL statement and is optional.
chain = prompt_template | kinetica_llm | KineticaSqlOutputParser(kdbc=kdbc)

================================ System Message ================================

CREATE TABLE nyse.prices AS
(
   t DATETIME NOT NULL COMMENT 'timestamp',
   s VARCHAR (8, dict, shard_key) NOT NULL COMMENT 'symbol',
   type VARCHAR (4, dict) NOT NULL,
   ap REAL  COMMENT 'ask price',
   as INTEGER,
   bp REAL  COMMENT 'bid price',
   bs INTEGER  COMMENT 'bid size',
   lp REAL  COMMENT 'sale price',
   ls INTEGER  COMMENT 'sale size'
);
COMMENT ON TABLE nyse.prices IS 'Stock prices including ask, bid, and sale price and size';
-- When querying table nyse.prices the following rules apply:
-- * when I ask about stock prices, use the nyse.prices table
-- * when I ask about stock prices today, filter on all results that occurred between now and an interval of 1 day
-- * all stock symbols are in lower case
-- * when I ask about today I mean that the timestamp should be greater than or equal to now minus an interval of 1 day
-- * when I ask about any column, make sure there are no null value

# Overview
### Setup and expectations
- Python code in the UDF folder is not intended to be run independently; it requires Kinetica to function properly
- Python code in the UDF folder is included for illustrative purposes
- Python code in the kinetica folder contains helper code and is not intended to be run independently
### Data Flow
- Data is streamed via websocket to Kinetica in JSON format
- Create delta table (just new quotes since last refresh)
- Invoke embedding model (UDF) over the delta table on every new record
- Embedding model (catch22) inserts 22 dimensional vector embedding into vector table for 5 minute windows every 5 seconds for every observed stock symbol
### Reference code
- if you want to dive deeper on the UDF that creates the vector embeddings see [./UDF/nyse_vector_embedding.py]()
- if you want to dive deeper on the data setup see [./kinetica/setup.py]()


In [11]:
response: SqlResponse = chain.invoke({"input": '''show me a sample of the vector data'''})
print(response.sql)
display(HTML(response.dataframe.to_html(index=False)))

GPUdbException: '[ERROR]: SqlEngine: Syntax error near ";" at line 3, column 9. (S/SDc:1454)'

In [10]:
response: SqlResponse = chain.invoke({"input": '''how many rows have flowed into the prices data in the last 15 minutes?'''})
display(HTML(response.dataframe.to_html(index=False)))

EXPR_0
0


### Some questions, including vector similarity search, without the SQL fuss

In [ ]:
from IPython.display import display, HTML

# Here you must ask a question relevant to the LLM context provided in the prompt template.
response: SqlResponse = chain.invoke({"input": '''what stock has traded with the highest volume today?'''})
display(HTML(response.dataframe.to_html(index=False)))
    

In [ ]:
response: SqlResponse = chain.invoke({"input": '''find all sofi stock trades between 2024-01-29 14:25:00 and 2024-01-29 14:35:00 where the price is recorded'''})
response.dataframe.plot.line(x='t', y='lp')

In [ ]:
response: SqlResponse = chain.invoke({"input": '''find similar patterns to sofi at 2024-01-29 14:25:00.000'''})
display(HTML(response.dataframe.to_html(index=False)))

In [ ]:
response: SqlResponse = chain.invoke({"input": '''find all qqq stock trades between 2024-01-22 16:00:00 and 2024-01-22 16:05:00 where the price is recorded'''})
response.dataframe.plot.line(x='t', y='lp')

### Get Kinetica LLM and Nemo talking to each other
- Helper code is in [./kinetica/kineai.py]() and can be referenced to understand how the two LLM's are connected to each other
- The intent with this exercise is to create a "Man in the Middle" RAG that provides us with the best features of two different LLM's, one for generating accurate SQL, then other to carry on a conversation and summarize tabular result sets when appropriate

In [ ]:
import importlib
import kinetica.kineai

importlib.reload(kinetica.kineai)
kineticallm = kinetica.kineai.KineticaLLM('nyse.nyse_vector_ctxt')

### The Nemo context object
- This is how we adjust how the LLM will react to our specific situation.
- Many samples are provided so that Nemo will be able to take in plain english, pass it to Kinetica's LLM, then summarize the results
- Note that small changes can have large effects.
- Feel free to experiment with this context to see what the effects are

In [ ]:
system = """ KineticAI is a cheerful AI assistant for engaging in a conversation between an LLM using the Nemo framework and the Kinetica LLM.  The Kinetica
LLM is designed to translate natural language questions into SQL queries. 

In addition to responding with  natural language it is able to ask questions to a database AI named SqlAssist that can query and summarize the logs. 
If it responds with a "KineticaLLM |  question" where question is sent to the SqlAssist AI. The SqlAssist AI will respond with an answer 
to the question in JSON format to the question made to SqlAssist by KineticAI.

when presented with a question, you should prefix your response with "KineticaLLM | "
if a sentence ends in a "?", you should prefix your response with "KineticaLLM | "

Consider the following example where a user asks KineticAI a question and KineticAI asks a followup question to SqlAssist. KineticAI uses the response from 
SqlAssist to answer the user's question.

user: what is the weather like today?
assistant: KineticaLLM | what is the weather like today?
user: KineticaLLM | [{"EXPR_0": 5.4}]
assistant: The answer is 5.4
"""

context0 = [dict(role="system", content=system),
            dict(role="user", content="what is the stock price today?"),
            dict(role="assistant", content="KineticaLLM | what is the stock price today?"),
            dict(role="user", content="how many rows of data are you storing?"),
            dict(role="assistant", content="KineticaLLM | how many rows of data are you storing?"),
            dict(role="user", content="what is the average number of telemetry rows per 5 second increment?"),
            dict(role="assistant", content="KineticaLLM | what is the average number of telemetry rows per 5 second increment?"),
            dict(role="user", content="find me top stock prices today"),
            dict(role="assistant", content="KineticaLLM | find me top stock prices today"),
            dict(role="user", content='KineticaLLM | [{"EXPR_0": 5.4}]'),
            dict(role="assistant", content='The answer is 5.4'),
            dict(role="user", content='KineticaLLM | [{"ts_bkt": "2024-02-07 19:35:00.000", "symbol": "aapl", "d1": -6467.2607421875}, {"ts_bkt": "2024-02-07 19:30:00.000", "symbol": "aapl", "d1": -6406.75341796875}, {"ts_bkt": "2024-02-07 19:45:00.000", "symbol": "tsla", "d1": -6331.88671875}, {"ts_bkt": "2024-02-07 19:40:00.000", "symbol": "tsla", "d1": -6128.375}]'),
            dict(role="assistant", content='It looks like aapl and tsla might be good choices')]

In [ ]:
question = 'what stock symbol other than Apple has the highest price within the last 15 minutes?'
kineticallm.chat(context0, question)

In [ ]:
question = 'find potential buying opportunities over the next 30 min?'
kineticallm.chat(context0, question)

In [ ]:
question = '''show me sale prices for QQQ for the last 12 hours'''
response: SqlResponse = chain.invoke({"input": question})
response.dataframe.plot.line(x='t', y='lp')